In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datasets import load_dataset
import evaluate
from collections import namedtuple
from tqdm import tqdm

In [2]:
dataset = load_dataset("e2e_nlg_cleaned")
dataset

DatasetDict({
    train: Dataset({
        features: ['meaning_representation', 'human_reference'],
        num_rows: 33525
    })
    validation: Dataset({
        features: ['meaning_representation', 'human_reference'],
        num_rows: 4299
    })
    test: Dataset({
        features: ['meaning_representation', 'human_reference'],
        num_rows: 4693
    })
})

In [4]:
class Llama31Instruct:
    def __init__(self, args):
        self.tokenizer = AutoTokenizer.from_pretrained(args.model_id)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "left"
        self.model = AutoModelForCausalLM.from_pretrained(args.model_id, device_map='auto', torch_dtype=torch.bfloat16)
        self.args = args
        self.terminators = [
            self.tokenizer.eos_token_id,
            self.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
        self.system_prompt = "You are a helpful assistant whose job is to generate natural english text from \
the attributes given by a user for a restaurant. Don't include any additional information."

    def get_batch_responses(self, batch_inputs):
        messages_batch = [[{"role": "system", "content": self.system_prompt}, 
                           {"role": "user", "content": inp+'\nConvert above attributes into free text.'}] 
                           for inp in batch_inputs]
        input_ids = self.tokenizer.apply_chat_template(
            messages_batch, 
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True
        ).to(self.model.device)
        attention_mask = input_ids.ne(self.tokenizer.pad_token_id).long()

        outputs = self.model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=self.args.max_new_tokens,
            eos_token_id=self.terminators,
            do_sample=True,
            temperature=self.args.temperature,
            top_p=self.args.top_p
        )

        responses = []
        prompt_len = input_ids.size(1)
        for output in outputs:
            response = self.tokenizer.decode(output[prompt_len:], skip_special_tokens=True)
            responses.append(response)
        return responses
    
Args = namedtuple('Args', ['model_id', 'max_new_tokens', 'temperature', 'top_p'])
args = Args(model_id="meta-llama/Meta-Llama-3-8B-Instruct", max_new_tokens=50, temperature=0.2, top_p=0.9)
model = Llama31Instruct(args)


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.39s/it]


In [10]:
outputs = []
split = 'test'
batch_size = 128
inputs = [eg['meaning_representation'] for eg in dataset[split]]
for i in tqdm(range(0, len(inputs), batch_size)):
    outputs += model.get_batch_responses(inputs[i:i+batch_size])

100%|██████████| 37/37 [02:33<00:00,  4.14s/it]


In [12]:
gt = [eg['human_reference'] for eg in dataset[split]]

In [14]:
bleu_metric = evaluate.load("bleu")
# rouge_metric = evaluate.load("rouge") # errors

In [15]:
bleu_metric.compute(predictions=outputs, references=gt)

{'bleu': 0.10481619723235527,
 'precisions': [0.3494322027525006,
  0.14500775305244348,
  0.07218957326023707,
  0.0329977990678405],
 'brevity_penalty': 1.0,
 'length_ratio': 1.6037614479725328,
 'translation_length': 199455,
 'reference_length': 124367}